# Snow Density Calculations

This notebook demonstrates methods for calculating density from common snow pit measurements (hand hardness and grain form) using the "Geldsetzer" method for calculating density. The analysis the local snowpyt_mechparams package and snowpylot for CAAML parsing.


In [1]:
# Import Libraries
import os
import sys

import numpy as np
import pandas as pd

# Import snowpylot for CAAML parsing
from snowpylot import caaml_parser

# Add the src directory to the path to import snowpyt_mechparams
sys.path.append('../src')
from snowpilot_utils import convert_grain_form
from snowpyt_mechparams import density


Parse Snowpit Files

In [2]:
all_pits = []
failed_files = []

folder_path = 'data'

xml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')]

for file in xml_files:
    try:
        file_path = os.path.join(folder_path, file)
        pit = caaml_parser(file_path)
        all_pits.append(pit)
    except Exception as e:
        failed_files.append((file, str(e)))
        print(f"Warning: Failed to parse {file}: {e}")

print(f"Successfully parsed {len(all_pits)} files")
print(f"Failed to parse {len(failed_files)} files")


Successfully parsed 50278 files
Failed to parse 0 files


Collect relevant data for each pit and convert to form needed to implement Geldsetzer

In [ ]:
# Collect relevant data from each snowpit
pit_info = []
layer_info = []

for pit in all_pits:
    pit_dict = {
        'pit_id': pit.core_info.pit_id,
        'layer_count': len(pit.snow_profile.layers),
    }
    pit_info.append(pit_dict)

    for layer in pit.snow_profile.layers:
        # Create base layer dictionary
        layer_dict = {
            'pit_id': pit.core_info.pit_id,
            'hand_hardness': layer.hardness,
            'grain_form_primary': layer.grain_form_primary,
        }

        # Add geldsetzer grain form conversion if grain form data exists
        if layer.grain_form_primary:
            layer_dict['geldsetzer_grain_form'] = convert_grain_form(layer.grain_form_primary, 'geldsetzer')
        else:
            layer_dict['geldsetzer_grain_form'] = None

        layer_info.append(layer_dict)

# Create a dataframe from the pit and layer info
pit_df = pd.DataFrame(pit_info)
layer_df = pd.DataFrame(layer_info)


In [10]:
# Dataset Summary
print("=== Dataset Summary ===")
print(f"Total number of snowpits: {len(all_pits)}")
print(f"Total number of layers: {len(layer_df)}")

# Filter layers with both hand hardness and grain form data
layers_with_data = layer_df[layer_df['hand_hardness'].notna() & layer_df['grain_form_primary'].notna()]
print(f"Layers with both hand hardness and grain form data: {len(layers_with_data)}, "
      f"{len(layers_with_data)/len(layer_df)*100:.1f}%")
geldsetzer_layers = layer_df[layer_df['geldsetzer_grain_form'].notna()]
print(f"Layers with Geldsetzer grain forms: {len(geldsetzer_layers)}, "
      f"{len(geldsetzer_layers)/len(layer_df)*100:.1f}%")


=== Dataset Summary ===
Total number of snowpits: 50278
Total number of layers: 371429
Layers with both hand hardness and grain form data: 279331, 75.2%
Layers with Geldsetzer grain forms: 221734, 59.7%


Implement Geldsetzer Method

In [6]:
# Calculate density using Geldsetzer method for each layer
def calculate_layer_density(row):
    try:
        # Only calculate if we have a mapped grain form
        if pd.isna(row['geldsetzer_grain_form']):
            return np.nan

        return density.calculate_density(
            method='geldsetzer',
            hand_hardness=row['hand_hardness'],
            grain_form=row['geldsetzer_grain_form']
        )
    except ValueError as e:
        # Return NaN for cases where density cannot be calculated
        return np.nan

# Add density column to the dataframe
layers_with_data = layers_with_data.copy()  # Avoid SettingWithCopyWarning
layers_with_data['density'] = layers_with_data.apply(calculate_layer_density, axis=1)

# Show summary of calculated densities
print("=== Density Calculation Results ===")
print(f"Layers with data and calculated density: {layers_with_data['density'].notna().sum()}")
print(f"Layers with data where density could not be calculated: {layers_with_data['density'].isna().sum()}")
print(f"Success rate of layers with data: {layers_with_data['density'].notna().sum()/len(layers_with_data)*100:.1f}%")
print(f"Success rate of all layers: {layers_with_data['density'].notna().sum()/len(layer_df)*100:.1f}%")


=== Density Calculation Results ===
Layers with data and calculated density: 200676
Layers with data where density could not be calculated: 78655
Success rate of layers with data: 71.8%
Success rate of all layers: 54.0%
